In [1]:
import time
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        exec_time = (te - ts) #int((te - ts) * 1000)
        print(method.__name__,' exec_time:',exec_time)
        return result
    return timed

import logging

my_logger = logging.getLogger('execution_logger')
my_logger.setLevel(logging.INFO)
my_logger_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
my_logger_handler = logging.FileHandler('execution_logger.log')


my_logger_handler.setFormatter(my_logger_format)
my_logger.addHandler(my_logger_handler)

#my_logger.info(f""" my_logger test """)
#my_logger.warning('This is a warning')
#my_logger.error('This is an error')

In [2]:
import datetime
import cx_Oracle
import getpass


class OracleConnect:
    def __init__(self):
        my_logger.info(f"""###OracleConnect :: __init__###' """)
        self.debug = False
        self.user = ''
        self.password = ''
        self.get_credentials()
        self.dsn_tns = self.select_database()
        self.engine_connection = self.connection()

        # defaults
        # xxxA
        self.foap_approvers = ['AMUSIAL1','EOBRIEN3']
        # xxxB
        self.acct_approvers = ['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']
        # Bxxx
        self.budget_approvers = ['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']




    def connection(self):
        try:
            _connection = cx_Oracle.connect(self.user, self.password,
                                            dsn=self.dsn_tns, #dsn_tns_prep, #'AVCPROD', #dsn_tns, #'AVCPREP',
                                            encoding="UTF-8")
            my_logger.info(f""" '###OracleConnect :: Connection Established ###', 'self.user:', {self.user}, 'self.dsn_tns:', {self.dsn_tns} """)
            return _connection
        except Exception as e:
            print(e)
            self.get_credentials()

        if self.debug == True:
            print('### ##### connection #######')
            print('## DB Connection as:', self.user, 'TNS string:',self.dsn_tns)
            print('## DB Connection object:', _connection)
            print('## if error:', (e if e else 'no error'))


    def get_credentials(self):
        self.user = getpass.getpass("Username: ")
        self.password = getpass.getpass("Password: ")

        if self.debug == True:
            print('### ##### get_credentials #######')
            print('## DB Connection as:', self.user, 'password:',self.password)


    def select_database(self):
        _db = input('which database to use (PROD,PREP):')
        if _db.upper() == 'PREP':
            _dsn_tns = cx_Oracle.makedsn('bandb-at1.test.avc.edu', '1521', 'avcprep')
        elif _db.upper() == 'PROD':
            print('not available yet')
            #_dsn_tns = cx_Oracle.makedsn('bandb-at1.test.avc.edu', '1521', 'avcprep')
        else:
            print("Ex. 'bandb-at1.test.avc.edu', '1521', 'avcprep'")
            _dsn_tns = cx_Oracle.makedsn(input('enter dns_tns string:') )

        if self.debug == True:
            print('### ##### select_database #######')
            print('## DB TNS String:', _dsn_tns)

        return _dsn_tns

    # Generic DB Query
    def db_query(self,_select,_from,_where,_order_by,_group_by,_max_records_returned):
        _select = f"""select  {_select}"""
        _from = f"""from {_from}"""
        _where = (f"""where {_where}""" if _where else '')
        _order_by = (f"""order by {_order_by}""" if _order_by else '')
        _group_by = (f"""group by {_group_by}"""  if _group_by else '')
        _max_records_returned = (f"""fetch next {_max_records_returned} rows only""" if _group_by else '')
        _query = f"""{_select} {_from} {_where} {_order_by} {_group_by} {_max_records_returned}"""
        #print(_query)
        app_data = self.execute_sql_query(_query)

        if self.debug == True:
            print('### ##### db_query #######')


        return app_data


    @timeit
    def execute_sql_query(self,_query):
        print(_query)
        with self.engine_connection.cursor() as _cur:
            _cur.execute(_query)
            app_data = _cur.fetchall()

        if self.debug == True:
            print('### ##### execute_sql_query #######')

        my_logger.info(f"""'execute_sql_query', '_query:', {_query}, 'app_data:', {app_data}""")

        return app_data


    def db_update(self, my_update, my_set, my_where):
        _update = f"""update {my_update}"""
        _set = f"""set {my_set}"""
        _where = f"""where {my_where}"""
        update_statement = f"""{_update} {_set} {_where}"""
        _result = self.execute_sql_update(update_statement)

        if self.debug == True:
            print('### ##### db_update #######')

        return _result


    @timeit
    def execute_sql_update(self,update_statement):
        print(update_statement)
        with self.engine_connection.cursor() as _cur:
            try:
                _cur.execute(update_statement)
                _result = self.engine_connection.commit()
            except Exception as e:
                self.engine_connection.rollback()
                return e

        if self.debug == True:
            print('### ##### execute_sql_update #######')

        my_logger.info(f"""'###OracleConnect :: execute_sql_update', 'update_statement:', {update_statement}, '_result:', {_result}""")

        return _result


    def db_insert(self, my_insert, my_values):
        _insert = f"""into {my_insert}"""
        _value = f"""values({my_values})"""
        insert_statement = f"""insert {_insert} {_value}"""
        _result = self.execute_sql_insert(insert_statement)

        if self.debug == True:
            print('### ##### db_insert #######')

        return _result


    @timeit
    def execute_sql_insert(self,insert_statement):
        print(insert_statement)
        with self.engine_connection.cursor() as _cur:
            try:
                _cur.execute(insert_statement)
                _result = self.engine_connection.commit()
            except Exception as e:
                self.engine_connection.rollback()
                return e

        if self.debug == True:
            print('### ##### execute_sql_insert #######')

        my_logger.info(f"""'execute_sql_insert', 'insert_statement:', {insert_statement}, '_result:', {_result}""")

        return _result


    def db_delete(self, my_from, my_where):
        _from = f"""from {my_from}"""
        _where = f"""where {my_where}"""
        delete_statement = f"""delete {_from} {_where}"""
        print('delete_statement',delete_statement)
        _result = self.execute_sql_delete(delete_statement)

        if self.debug == True:
            print('### ##### db_delete #######')


    @timeit
    def execute_sql_delete(self,delete_statement):
        print(delete_statement)
        with self.engine_connection.cursor() as _cur:
            try:
                _cur.execute(delete_statement)
                _result = self.engine_connection.commit()
            except Exception as e:
                self.engine_connection.rollback()
                return e

        if self.debug == True:
            print('### ##### execute_sql_delete #######')

        my_logger.info(f"""'execute_sql_delete', 'delete_statement:', {delete_statement}, '_result:', {_result}""")

        return _result

    # Use specific methods
    ## Approval Queue Methods
    def query_foraqus(self, my_where='',my_order_by='',my_group_by='',my_max_records_returned = ''):
        select = f"""FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
        FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
        from_ = f"""FORAQUS"""
        x =  self.db_query(select,from_,my_where,my_order_by,my_group_by,my_max_records_returned)

        if self.debug == True:
            print('### ##### query_foraqus #######')

        return x


    def query_ftvappq(self, my_where='',my_order_by='',my_group_by='',my_max_records_returned = ''):
        select = f"""FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID"""
        from_ = f"""FTVAPPQ"""
        x =  self.db_query(select,from_,my_where,my_order_by,my_group_by,my_max_records_returned)

        if self.debug == True:
            print('### ##### query_ftvappq #######')

        return x


    def query_foraqrc(self, my_where='',my_order_by='',my_group_by='',my_max_records_returned = ''):
        select = f"""FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE"""
        from_ = f"""FORAQRC"""
        x =  self.db_query(select,from_,my_where,my_order_by,my_group_by,my_max_records_returned)

        if self.debug == True:
            print('### ##### query_foraqrc #######')

        return x


    def insert_into_ftvappq(self, _my_value):
        my_insert = """FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID)"""
        x = self.db_insert(my_insert, _my_value)
        print(x)

        if self.debug == True:
            print('### ##### insert_into_ftvappq #######')


    def insert_into_foraqus(self, _my_value):
        my_insert = """FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID)"""
        x = self.db_insert(my_insert, _my_value)
        print(x)

        if self.debug == True:
            print('### ##### insert_into_foraqus #######')


    def insert_into_foraqrc(self, _my_value):
        my_insert = """FORAQRC(FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE) """
        x = self.db_insert(my_insert, _my_value)
        print(x)

        if self.debug == True:
            print('### ##### insert_into_foraqrc #######')


    def get_next_queue_number(self):
        _select = "max(to_number(substr(FTVAPPQ_QUEUE_ID,0,3)))"
        _from = "FTVAPPQ"
        _where = "substr(FTVAPPQ_QUEUE_ID,0,1) in ('0','1','2','3','4','5','6','7','8','9')"
        x = zxc.db_query(_select,_from,_where,_order_by = '',_group_by = '',_max_records_returned = '')

        if self.debug == True:
            print('### ##### get_next_queue_number #######')
            print('current max queue num:',x[0][0], 'next queue to use:', (x[0][0]+1))

        my_logger.info(f"""'get_next_queue_number', 'current max queue num:',{x[0][0]}, 'next queue to use:', {x[0][0] + 1}""")

        return (x[0][0]+1)


    def get_approvers_queues(self,approver_current_queue_list):
        x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_current_queue_list}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
        current_approver_queues = [y[0] for y in x]

        if self.debug == True:
            print('### ##### get_approvers_queues #######')

        return approver_current_queue_list, current_approver_queues


    # Procedures

    ## Build Queue Chain
    def build_a_new_queue_chain(self,queue_chain_number,queue_description,queues_to_build,orgns_to_route):
        # add budget queue Bxxx to queues_to_build for processing
        budget_queue = ('budget', str(queue_chain_number)+queues_to_build[2][0], 0, self.budget_approvers)
        queues_to_build.append(budget_queue)

        # build each queue in queue chain
        for que in queues_to_build:
            # build the queue
            the_que = str(queue_chain_number)+que[0] if  que[0] != 'budget' else 'B'+str(queue_chain_number)
            exit_que = (que[1] if que[1] in ['VPAS','EXBS','PRES'] else str(queue_chain_number)+que[1])
            build_queue = f"""'{the_que}','{queue_description}',{que[2]},'{exit_que}', null, sysdate, 'RWALDEN'"""
            print(build_queue)
            self.insert_into_ftvappq(build_queue)


            # add approvers
            for approver in que[3]:
                insert_approvers = f"""'{the_que}', '{approver}', 10, {que[2]}, sysdate, null, null, sysdate, 'RWALDEN'"""
                print(insert_approvers)
                self.insert_into_foraqus(insert_approvers)


            # route orgns to Accounting queue in queue chain
            if que[0] == 'A':
                for orgn in orgns_to_route:
                    req_route_orgn = f"""'{the_que}', 1, 'REQG', 'A', '', '', '{orgn}', '', '', sysdate, 'RWALDEN', ''"""
                    po_route_orgn = f"""'{the_que}', 2, 'CORG', 'A', '', '', '{orgn}', '', '', sysdate, 'RWALDEN', ''"""
                    print(req_route_orgn)
                    print(po_route_orgn)
                    self.insert_into_foraqrc(req_route_orgn)
                    self.insert_into_foraqrc(po_route_orgn)


            # route orgns to Budget queue in queue chain
            if que[0] == 'budget':
                for orgn in orgns_to_route:
                    jv_route_orgn = f"""'{the_que}', 20, 'DBRG', 'A', '', '', '{orgn}', '', '', sysdate, 'RWALDEN', ''"""
                    print(jv_route_orgn)
                    self.insert_into_foraqrc(jv_route_orgn)

        if self.debug == True:
            print('### ##### build_a_new_queue_chain #######')
    ### EX: Build new queue chain start
    ### EX: queue_chain_number = 998
    ### EX: queue_description = 'Test Queue Desc'
    ### EX: queues_to_build = [('A','B',0,['AMUSIAL1','EOBRIEN3']),('B','G',.01,['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']),('G','EXBS',4999.99,['SMILLER66'])]
    ### EX: orgns_to_route = ['12345','67890','19283']
    ### EX: build_a_new_queue_chain(queue_chain_number, queue_description, queues_to_build, orgns_to_route)
    ### EX: Build new queue chain end

    ## Insert Proxy
    ### Set Proxy date format '28-JUL-22' == July 28 2022


    ## Set Proxy Approver
    def set_proxy_approver(approver_to_proxy,proxy_approver,date_from,date_to):
        x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_proxy}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
        #print(x)
        for y in x:
            insert_term_record = f"""'{y[0]}', '{proxy_approver}', {y[2]}, {y[3]}, '{date_from}', null, '{date_to}', sysdate, 'RWALDEN' """
            zxc.insert_into_foraqus(insert_term_record)

        if self.debug == True:
            print('### ##### set_proxy_approver #######')
    #### EX: set_proxy_approver(approver_to_proxy = 'EKNUDSON',proxy_approver = 'RWALDEN',date_from = '22-JUL-22',date_to = '28-JUL-22')

    ## Replace Approver in Queues

    # Replace Approver
    def replace_approver(self,approver_to_place, new_approver, from_queues = ()):
        if (len(from_queues) < 1) or (from_queues == ''):
            x = self.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
        elif type(from_queues) == str:
            x = self.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID in ('{from_queues}')")
        else:
            x = self.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID in {from_queues}")
        #x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
        for y in x:
            print(y[0], y[1], y[2], y[3], y[4], y[5], y[6])
            my_update = 'FORAQUS'
            my_set = 'FORAQUS_NCHG_DATE = sysdate'
            my_where = f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID = '{y[0]}'"
            zxc.db_update(my_update, my_set, my_where)
            # insert term record
            insert_term_record = f"""'{y[0]}', '{y[1]}', {y[2]}, {y[3]}, sysdate, null, sysdate, sysdate, 'RWALDEN' """
            zxc.insert_into_foraqus(insert_term_record)
            # insert new approver
            insert_term_record = f"""'{y[0]}', '{new_approver}', {y[2]}, {y[3]}, sysdate, null, null, sysdate, 'RWALDEN' """
            zxc.insert_into_foraqus(insert_term_record)

        if self.debug == True:
            print('### ##### replace_approver #######')
    ### EX: replace_approver(approver_to_place = 'BJESTER1', new_approver = 'RWALDEN')


    ## Term from queues
    ### Term Approver date format '28-JUL-22' == July 28 2022
    ### EX: from_queues = () or '000A' or ('000A') or ('000A','B000','000B') ; all queues , one queue, multiple queues
    def term_approver(self,approver_to_term, term_date, from_queues = ()):
        if (len(from_queues) < 1) or (from_queues == ''):
            x = self.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
        elif type(from_queues) == str:
            x = self.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID in ('{from_queues}')")
        else:
            x = self.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID in {from_queues}")

        for y in x:
            # update next change date in current active record
            my_update = 'FORAQUS'
            my_set = f"FORAQUS_NCHG_DATE = '{term_date}"
            my_where = f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID = '{y[0]}'"
            self.db_update(my_update, my_set, my_where)
            # insert term record
            insert_term_record = f"""'{y[0]}', '{y[1]}', {y[2]}, {y[3]}, sysdate, null, sysdate, sysdate, 'RWALDEN' """
            self.insert_into_foraqus(insert_term_record)

        if self.debug == True:
            print('### ##### term_approver #######')
    ### EX: term_approver('AMUSIAL1', '28-JUL-22')
    ### EX: term_approver('AMUSIAL1', '28-JUL-22', from_queues = ('005A'))
    ### EX: term_approver('AMUSIAL1', '28-JUL-22', from_queues = ('005A','006A'))


In [3]:
zxc = OracleConnect()
conn = zxc.engine_connection

ORA-01035: ORACLE only available to users with RESTRICTED SESSION privilege


In [4]:
x =1/0

ZeroDivisionError: division by zero

In [ ]:
conn.close()

In [6]:
# Current work to do
queue_chain_number = zxc.get_next_queue_number()
queue_description = 'Kaiser Sim Collaboration'
queues_to_build = [('A','B',0,['AMUSIAL1','EOBRIEN3']),('B','G',.01,['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']),('G','VPAS',.05,['SMILLER66'])]
orgns_to_route = ['62404']
zxc.build_a_new_queue_chain(queue_chain_number, queue_description, queues_to_build, orgns_to_route)

queue_chain_number = zxc.get_next_queue_number()
queue_description = 'LACOE AB130 Outreach Support'
queues_to_build = [('A','B',0,['AMUSIAL1','EOBRIEN3']),('B','G',.01,['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']),('G','EXBS',4999.99,['CVALIOTIS'])]
orgns_to_route = ['62719']
zxc.build_a_new_queue_chain(queue_chain_number, queue_description, queues_to_build, orgns_to_route)

queue_chain_number = zxc.get_next_queue_number()
queue_description = 'Economic Workforce Development'
queues_to_build = [('A','B',0,['AMUSIAL1','EOBRIEN3']),('B','G',.01,['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']),('G','EXBS',4999.99,['CHART8'])]
orgns_to_route = ['62408']
zxc.build_a_new_queue_chain(queue_chain_number, queue_description, queues_to_build, orgns_to_route)

# term nick
x,y = zxc.get_approvers_queues('NVALSAMIDES')
print(x,y)

zxc.term_approver('NVALSAMIDES', '28-JUL-22', from_queues = ())

x,y = zxc.get_approvers_queues('NVALSAMIDES')
print(x,y)

# revise our approvers for Human Resources org 11030
# Harmony will remain on all of the above.
# Add James Firth (jfirth@avc.edu) to org 11030 as an approver
# Remove Debby Hackenberg (dclarkhackenberg@avc.edu) from org 11030
# Debby Hackenberg will need to be assigned and the approver for her org 11035

replace_approver('DCLARKHACKENBERG', 'JFIRTH')


insert_into_foraqus("""'137E', 'DCLARKHACKENBERG', 10, 999.99, sysdate, null, null, sysdate, 'RWALDEN' """)


select  max(to_number(substr(FTVAPPQ_QUEUE_ID,0,3))) from FTVAPPQ where substr(FTVAPPQ_QUEUE_ID,0,1) in ('0','1','2','3','4','5','6','7','8','9')   
execute_sql_query  exec_time: 0.06592631340026855
'135A','Kaiser Sim Collaboration',0,'135B', null, sysdate, 'RWALDEN'
insert into FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID) values('135A','Kaiser Sim Collaboration',0,'135B', null, sysdate, 'RWALDEN')
execute_sql_insert  exec_time: 0.08616018295288086
None
'135A', 'AMUSIAL1', 10, 0, sysdate, null, null, sysdate, 'RWALDEN'
insert into FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID) values('135A', 'AMUSIAL1', 10, 0, sysdate, null, null, sysdate, 'RWALDEN')
execute_sql_insert  exec_time: 0.0686643123626709
None
'135A', 'EOBRIEN3', 10, 0, sysdate, null, null,

NameError: name 'replace_approver' is not defined

In [ ]:
zxc.replace_approver('DCLARKHACKENBERG', 'JFIRTH')

In [ ]:
zxc.replace_approver('JFIRTH','DCLARKHACKENBERG', '040C')

In [ ]:
zxc.get_next_queue_number()
x,y = zxc.get_approvers_queues('DCLARKHACKENBERG') #('NVALSAMIDES')
print(x,y)

In [ ]:
# Term Approver date format '28-JUL-22' == July 28 2022
# EX: from_queues = ('000A','B000','000B')
def term_approver(approver_to_term, term_date, from_queues = ()):
    if len(from_queues) < 1:
        x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
    elif type(from_queues) == str:
        x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID in ('{from_queues}')")
    else:
        x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID in {from_queues}")
    #print(x)
    for y in x:
        print(y[0], y[1], y[2], y[3], y[4], y[5], y[6])
        # update next change date in current active record
        my_update = 'FORAQUS'
        my_set = f"FORAQUS_NCHG_DATE = '{term_date}"
        my_where = f"FORAQUS_USER_ID_APPR = '{approver_to_term}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID = '{y[0]}'"
        zxc.db_update(my_update, my_set, my_where)
        # insert term record
        insert_term_record = f"""'{y[0]}', '{y[1]}', {y[2]}, {y[3]}, sysdate, null, sysdate, sysdate, 'RWALDEN' """
        zxc.insert_into_foraqus(insert_term_record)

In [ ]:
#zxc.term_approver('AMUSIAL1', '28-JUL-22')
#zxc.term_approver('WDUMAS1', '28-JUL-22', '')
zxc.term_approver('AMUSIAL1', '28-JUL-22', from_queues = ('009A'))
#zxc.term_approver('AMUSIAL1', '28-JUL-22', from_queues = ('007A','008A'))

In [ ]:
def get_next_queue_number():
    _select = "max(to_number(substr(FTVAPPQ_QUEUE_ID,0,3)))"
    _from = "FTVAPPQ"
    _where = "substr(FTVAPPQ_QUEUE_ID,0,1) in ('0','1','2','3','4','5','6','7','8','9')"
    x = zxc.db_query(_select,_from,_where,_order_by = '',_group_by = '',_max_records_returned = '')
    print('current max queue num:',x[0][0], 'next queue to use:', (x[0][0]+1))
    return (x[0][0]+1)

get_next_queue_number()

In [ ]:
def get_approvers_queues(approver_current_queue_list):
    x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_current_queue_list}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
    current_approver_queues = [y[0] for y in x]
    return approver_current_queue_list, current_approver_queues

x,y = get_approvers_queues('RWALDEN') #('NVALSAMIDES')
print(x,y)

In [ ]:
# Set Proxy date format '28-JUL-22' == July 28 2022
def set_proxy_approver(approver_to_proxy,proxy_approver,date_from,date_to):
    x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_proxy}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
    #print(x)
    for y in x:
        insert_term_record = f"""'{y[0]}', '{proxy_approver}', {y[2]}, {y[3]}, '{date_from}', null, '{date_to}', sysdate, 'RWALDEN' """
        zxc.insert_into_foraqus(insert_term_record)

set_proxy_approver(approver_to_proxy = 'EKNUDSON',proxy_approver = 'RWALDEN',date_from = '22-JUL-22',date_to = '28-JUL-22')

In [ ]:
# Replace Approver in Queues
def replace_approver(approver_to_place, new_approver):
    x = zxc.query_foraqus(my_where=f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
    #print(x)
    for y in x:
        print(y[0], y[1], y[2], y[3], y[4], y[5], y[6])
        my_update = 'FORAQUS'
        my_set = 'FORAQUS_NCHG_DATE = sysdate'
        my_where = f"FORAQUS_USER_ID_APPR = '{approver_to_place}' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null and FORAQUS_QUEUE_ID = '{y[0]}'"
        zxc.db_update(my_update, my_set, my_where)
        # insert term record
        insert_term_record = f"""'{y[0]}', '{y[1]}', {y[2]}, {y[3]}, sysdate, null, sysdate, sysdate, 'RWALDEN' """
        zxc.insert_into_foraqus(insert_term_record)
        # insert new approver
        insert_term_record = f"""'{y[0]}', '{new_approver}', {y[2]}, {y[3]}, sysdate, null, null, sysdate, 'RWALDEN' """
        zxc.insert_into_foraqus(insert_term_record)
        break

replace_approver(approver_to_place = 'BJESTER1', new_approver = 'RWALDEN')

In [ ]:
x = zxc.query_foraqus(my_where="FORAQUS_USER_ID_APPR = 'DCLARKHACKENBERG' and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null", my_order_by="FORAQUS_QUEUE_ID")
#print(x)
for y in x:
    print(y[0], y[1], y[2], y[3], y[4], y[5], y[6])

In [ ]:
x = zxc.query_foraqus(my_where="FORAQUS_USER_ID_APPR = 'BJESTER1'", my_order_by="FORAQUS_QUEUE_ID") # and FORAQUS_NCHG_DATE is null and FORAQUS_TERM_DATE is null")
#print(x)
for y in x:
    print(y[0], y[1], y[2], y[3], y[4], y[5], y[6])

In [ ]:
print(sysdate)

# Build a Queue

In [ ]:

# ################################
queue_chain_number = 998
queue_description = 'Test Queue Desc'
queues_to_build = [('A','B',0,['AMUSIAL1','EOBRIEN3']),('B','G',.01,['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']),('G','EXBS',4999.99,['SMILLER66'])]
orgns_to_route = ['12345','67890','19283']
zxc.build_a_new_queue_chain(queue_chain_number, queue_description, queues_to_build, orgns_to_route)

In [ ]:
print(zxc.query_ftvappq("FTVAPPQ_QUEUE_ID in ('998A','998B','998G','B998')"))
print(zxc.query_foraqus("FORAQUS_QUEUE_ID in ('998A','998B','998G','B998')"))
print(zxc.query_foraqrc("FORAQRC_QUEUE_ID in ('998A','998B','998G','B998')"))

In [ ]:
print(zxc.db_delete('FTVAPPQ', "FTVAPPQ_QUEUE_ID in ('998A','998B','998G','B998')"))
print(zxc.db_delete('FORAQUS', "FORAQUS_QUEUE_ID in ('998A','998B','998G','B998')"))
print(zxc.db_delete('FORAQRC', "FORAQRC_QUEUE_ID in ('998A','998B','998G','B998')"))

# Build Test Queue 999

In [ ]:
def build_a_new_queue_chain(queue_chain_number,queue_description,queues_to_build,orgns_to_route):
    # add budget queue Bxxx to queues_to_build for processing
    budget_queue = ('budget', str(queue_chain_number)+queues_to_build[2][0], 0, ['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK'])
    queues_to_build.append(budget_queue)

    # build each queue in queue chain
    for que in queues_to_build:
        # build the queue
        the_que = str(queue_chain_number)+que[0] if  que[0] != 'budget' else 'B'+str(queue_chain_number)
        exit_que = (que[1] if que[1] in ['VPAS','EXBS','PRES'] else str(queue_chain_number)+que[1])
        build_queue = f"""'{the_que}','{queue_description}',{que[2]},'{exit_que}', null, sysdate, 'RWALDEN'"""
        print(build_queue)
        zxc.insert_into_ftvappq(build_queue)


        # add approvers
        for approver in que[3]:
            insert_approvers = f"""'{the_que}', '{approver}', 10, {que[2]}, sysdate, null, null, sysdate, 'RWALDEN'"""
            print(insert_approvers)
            zxc.insert_into_foraqus(insert_approvers)


        # route orgns to Accounting queue in queue chain
        if que[0] == 'A':
            for orgn in orgns_to_route:
                req_route_orgn = f"""'{the_que}', 1, 'REQG', 'A', '', '', '{orgn}', '', '', sysdate, 'RWALDEN', ''"""
                po_route_orgn = f"""'{the_que}', 2, 'CORG', 'A', '', '', '{orgn}', '', '', sysdate, 'RWALDEN', ''"""
                print(req_route_orgn)
                print(po_route_orgn)
                zxc.insert_into_foraqrc(req_route_orgn)
                zxc.insert_into_foraqrc(po_route_orgn)


        # route orgns to Budget queue in queue chain
        if que[0] == 'budget':
            for orgn in orgns_to_route:
                jv_route_orgn = f"""'{the_que}', 20, 'DBRG', 'A', '', '', '{orgn}', '', '', sysdate, 'RWALDEN', ''"""
                print(jv_route_orgn)
                zxc.insert_into_foraqrc(jv_route_orgn)
# ################################
queue_chain_number = 998
queue_description = 'Test Queue Desc'
queues_to_build = [('A','B',0,['AMUSIAL1','EOBRIEN3']),('B','G',.01,['KJANISZEWSKI','UGOFF', 'BJESTER1', 'LNOWAK']),('G','EXBS',4999.99,['SMILLER66'])]
orgns_to_route = ['12345','67890','19283']
build_a_new_queue_chain(queue_chain_number, queue_description, queues_to_build, orgns_to_route)

In [ ]:
print(zxc.query_ftvappq("FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.query_foraqus("FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.query_foraqrc("FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
print(zxc.insert_into_ftvappq("""'999A', '999-Test Orgn- Test Que', 0, '999B', null, sysdate, 'RWALDEN' """))
print(zxc.insert_into_ftvappq("""'999B', '999-Test Orgn- Test Que', 100.01, '999G', null, sysdate, 'RWALDEN' """))
print(zxc.insert_into_ftvappq("""'999G', '999-Test Orgn- Test Que', 1000.01, 'EXBS', null, sysdate, 'RWALDEN' """))
print(zxc.insert_into_ftvappq("""'B999', '999-Test Orgn- Test Que', 10000.01, '999G', null, sysdate, 'RWALDEN' """))

print(zxc.query_ftvappq("FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
print(zxc.insert_into_foraqus("""'999A', 'RWALDEN', 10, 0, sysdate, null, null, sysdate, 'RWALDEN' """))
print(zxc.insert_into_foraqus("""'999B', 'RWALDEN', 10, 100.01, sysdate, null, null, sysdate, 'RWALDEN' """))
print(zxc.insert_into_foraqus("""'999G', 'RWALDEN', 10, 1000.02, sysdate, null, null, sysdate, 'RWALDEN' """))
print(zxc.insert_into_foraqus("""'B999', 'RWALDEN', 10, 10000.03, sysdate, null, null, sysdate, 'RWALDEN' """))

print(zxc.query_foraqus("FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
print(zxc.insert_into_foraqrc("""'999A', 1, 'REQG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """))
print(zxc.insert_into_foraqrc("""'999A', 2, 'CORG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """))
print(zxc.insert_into_foraqrc("""'B999', 20, 'DBRG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """))

print(zxc.query_foraqrc("FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
print(zxc.db_delete('FTVAPPQ', "FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.db_delete('FORAQUS', "FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.db_delete('FORAQRC', "FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

print(zxc.query_ftvappq("FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.query_foraqus("FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.query_foraqrc("FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
x = 1/0

# Query Tables

In [ ]:
def query_foraqus(my_where='',my_order_by='',my_group_by='',my_max_records_returned = ''):
    select = f"""FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
    FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
    from_ = f"""FORAQUS"""
    x =  zxc.db_query(select,from_,my_where,my_order_by,my_group_by,my_max_records_returned)
    return x
def query_ftvappq(my_where='',my_order_by='',my_group_by='',my_max_records_returned = ''):
    select = f"""FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID"""
    from_ = f"""FTVAPPQ"""
    x =  zxc.db_query(select,from_,my_where,my_order_by,my_group_by,my_max_records_returned)
    return x
def query_foraqrc(my_where='',my_order_by='',my_group_by='',my_max_records_returned = ''):
    select = f"""FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE"""
    from_ = f"""FORAQRC"""
    x =  zxc.db_query(select,from_,my_where,my_order_by,my_group_by,my_max_records_returned)
    return x

In [ ]:
print(query_ftvappq("FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))
print(query_foraqus("FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))
print(query_foraqrc("FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
def insert_into_ftvappq(_my_value):
    my_insert = """FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID)"""
    x = zxc.db_insert(my_insert, _my_value)
    print(x)

print(insert_into_ftvappq("""'999A', '999-Test Orgn- Test Que', 0, '999B', null, sysdate, 'RWALDEN' """))
print(insert_into_ftvappq("""'999B', '999-Test Orgn- Test Que', 100.01, '999G', null, sysdate, 'RWALDEN' """))
print(insert_into_ftvappq("""'999G', '999-Test Orgn- Test Que', 1000.01, 'EXBS', null, sysdate, 'RWALDEN' """))
print(insert_into_ftvappq("""'B999', '999-Test Orgn- Test Que', 10000.01, '999G', null, sysdate, 'RWALDEN' """))

print(query_ftvappq("FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
def insert_into_foraqus(_my_value):
    my_insert = """FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID)"""
    x = zxc.db_insert(my_insert, _my_value)
    print(x)

print(insert_into_foraqus("""'999A', 'RWALDEN', 10, 0, sysdate, null, null, sysdate, 'RWALDEN' """))
print(insert_into_foraqus("""'999B', 'RWALDEN', 10, 100.01, sysdate, null, null, sysdate, 'RWALDEN' """))
print(insert_into_foraqus("""'999G', 'RWALDEN', 10, 1000.02, sysdate, null, null, sysdate, 'RWALDEN' """))
print(insert_into_foraqus("""'B999', 'RWALDEN', 10, 10000.03, sysdate, null, null, sysdate, 'RWALDEN' """))

print(query_foraqus("FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
def insert_into_foraqrc(_my_value):
    my_insert = """FORAQRC(FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE) """
    x = zxc.db_insert(my_insert, _my_value)
    print(x)

print(insert_into_foraqrc("""'999A', 1, 'REQG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """))
print(insert_into_foraqrc("""'999A', 2, 'CORG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """))
print(insert_into_foraqrc("""'B999', 20, 'DBRG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """))

print(query_foraqrc("FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
print(zxc.db_delete('FTVAPPQ', "FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.db_delete('FORAQUS', "FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))
print(zxc.db_delete('FORAQRC', "FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

print(query_ftvappq("FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"))
print(query_foraqus("FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"))
print(query_foraqrc("FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"))

In [ ]:
# Query FORAQUS
select = f"""FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
from_ = f"""FORAQUS"""
where = f"""FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"""
order_by = f"""FORAQUS_QUEUE_ID"""
group_by = f""""""
max_records_returned = '' #10 #f"""10"""

x =  zxc.db_query(select,from_,where,order_by,group_by,max_records_returned)
#print(dir(x[0]))
for y in x:
    print(x.index(y), 'of', len(x)-1)
    print(y)

In [ ]:
# Query FTVAPPQ
select = f"""FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID"""
from_ = f"""FTVAPPQ"""
where = f"""FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"""
order_by = f"""FTVAPPQ_QUEUE_ID"""
group_by = f""""""
max_records_returned = '' #10 #f"""10"""
x = zxc.db_query(select,from_,where,order_by,group_by,max_records_returned)
#print(dir(x[0]))
for y in x:
    print(x.index(y), 'of', len(x)-1)
    print(y)

In [ ]:
# Query FORAQRC
select = f"""FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE"""
from_ = f"""FORAQRC"""
where = f"""FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"""
order_by = f"""FORAQRC_QUEUE_ID"""
group_by = f""""""
max_records_returned = '' #10 #f"""10"""
x = zxc.db_query(select,from_,where,order_by,group_by,max_records_returned)
#print(dir(x[0]))
for y in x:
    print(x.index(y), 'of', len(x)-1)
    print(y)

# Update Tables

In [ ]:
# Update FTVAPPQ
my_update = 'FTVAPPQ'
my_set = 'FTVAPPQ_ACTIVITY_DATE = sysdate'
my_where = "FTVAPPQ_QUEUE_ID = '000B'"
x = zxc.db_update(my_update, my_set, my_where)
print(x)

# Make New Test Queues

# Create Test Queues

In [ ]:
# Insert FTVAPPQ
my_insert = """FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID)"""
my_value = """'999A', '999-Test Orgn- Test Que', 0, '999B', null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FTVAPPQ
my_insert = """FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID)"""
my_value = """'999B', '999-Test Orgn- Test Que', 0, '999G', null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FTVAPPQ
my_insert = """FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID)"""
my_value = """'999G', '999-Test Orgn- Test Que', 0, 'EXBS', null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FTVAPPQ
my_insert = """FTVAPPQ(FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID)"""
my_value = """'B999', '999-Test Orgn- Test Que', 0, '999G', null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

# Insert Users into Test Queues

In [ ]:
# Insert FORAQUS
my_insert = """FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID)"""
my_value = """'999A', 'RWALDEN', 10, 0, sysdate, null, null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FORAQUS
my_insert = """FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID)"""
my_value = """'999B', 'RWALDEN', 10, 0, sysdate, null, null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FORAQUS
my_insert = """FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID)"""
my_value = """'999G', 'RWALDEN', 10, 0, sysdate, null, null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FORAQUS
my_insert = """FORAQUS(FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE,FORAQUS_ACTIVITY_DATE,FORAQUS_USER_ID)"""
my_value = """'B999', 'RWALDEN', 10, 0, sysdate, null, null, sysdate, 'RWALDEN' """
x = zxc.db_insert(my_insert, my_value)
print(x)

# Insert Routes for test queues FORAQRC

In [ ]:
# Insert FORAQRC REQ 999A
my_insert = """FORAQRC(FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE) """
my_value = """'999A', 1, 'REQG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FORAQRC PO 999A
my_insert = """FORAQRC(FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE) """
my_value = """'999A', 2, 'CORG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """
x = zxc.db_insert(my_insert, my_value)
print(x)

In [ ]:
# Insert FORAQRC JV B999
my_insert = """FORAQRC(FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE) """
my_value = """'B999', 20, 'DBRG', 'A', '', '', '14020', '', '', sysdate, 'RWALDEN', '' """
x = zxc.db_insert(my_insert, my_value)
print(x)

# Delete test queues

In [ ]:
# Delete test queues
my_from = 'FTVAPPQ'
my_where = "FTVAPPQ_QUEUE_ID in ('999A','999B','999G','B999')"
x = zxc.db_delete(my_from, my_where)
print(x)

my_from = 'FORAQUS'
my_where = "FORAQUS_QUEUE_ID in ('999A','999B','999G','B999')"
x = zxc.db_delete(my_from, my_where)
print(x)

my_from = 'FORAQRC'
my_where = "FORAQRC_QUEUE_ID in ('999A','999B','999G','B999')"
x = zxc.db_delete(my_from, my_where)
print(x)

# Queue Tables
FIMSMGR.FORAQRC FORAQRC,
FIMSMGR.FORAQUS FORAQUS,
FIMSMGR.FTVAPPQ FTVAPPQ

# FORAQRC fields
### FORAQRC_DOC_TYPE is a number REQ = 1, PO = 2, JV = 20
FORAQRC_QUEUE_ID, FORAQRC_DOC_TYPE, FORAQRC_RULE_GROUP, FORAQRC_COAS_CODE, FORAQRC_FTYP_CODE, FORAQRC_FUND_CODE, FORAQRC_ORGN_CODE, FORAQRC_ATYP_CODE, FORAQRC_ACCT_CODE, FORAQRC_ACTIVITY_DATE, FORAQRC_USER_ID, FORAQRC_PROG_CODE

--FORAQRC_SURROGATE_ID, FORAQRC_VERSION, FORAQRC_DATA_ORIGIN, FORAQRC_VPDI_CODE

FORAQRC_QUEUE_ID
FORAQRC_DOC_TYPE
FORAQRC_RULE_GROUP
FORAQRC_COAS_CODE
FORAQRC_FTYP_CODE
FORAQRC_FUND_CODE
FORAQRC_ORGN_CODE
FORAQRC_ATYP_CODE
FORAQRC_ACCT_CODE
FORAQRC_ACTIVITY_DATE
FORAQRC_USER_ID
FORAQRC_PROG_CODE

FORAQRC_SURROGATE_ID
FORAQRC_VERSION
FORAQRC_DATA_ORIGIN
FORAQRC_VPDI_CODE

# FORAQUS fields
FORAQUS(FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR, FORAQUS_QUEUE_LEVEL, FORAQUS_QUEUE_LIMIT, FORAQUS_EFF_DATE, FORAQUS_NCHG_DATE, FORAQUS_TERM_DATE, FORAQUS_ACTIVITY_DATE, FORAQUS_USER_ID, FORAQUS_SURROGATE_ID, FORAQUS_VERSION, FORAQUS_DATA_ORIGIN, FORAQUS_VPDI_CODE)

FORAQUS_QUEUE_ID
FORAQUS_USER_ID_APPR
FORAQUS_QUEUE_LEVEL
FORAQUS_QUEUE_LIMIT
FORAQUS_EFF_DATE
FORAQUS_NCHG_DATE
FORAQUS_TERM_DATE

FORAQUS_ACTIVITY_DATE, FORAQUS_USER_ID, FORAQUS_SURROGATE_ID, FORAQUS_VERSION, FORAQUS_DATA_ORIGIN, FORAQUS_VPDI_CODE

# FTVAPPQ fields

 FTVAPPQ_QUEUE_ID, FTVAPPQ_DESCRIPTION, FTVAPPQ_QUEUE_LIMIT, FTVAPPQ_NEXT_QUEUE_ID, FTVAPPQ_APPROVAL_REQ, FTVAPPQ_ACTIVITY_DATE, FTVAPPQ_USER_ID

--, FTVAPPQ_SURROGATE_ID, FTVAPPQ_DATA_ORIGIN, FTVAPPQ_VERSION, FTVAPPQ_VPDI_CODE

FTVAPPQ_QUEUE_ID
FTVAPPQ_DESCRIPTION
FTVAPPQ_QUEUE_LIMIT
FTVAPPQ_NEXT_QUEUE_ID
FTVAPPQ_APPROVAL_REQ
FTVAPPQ_ACTIVITY_DATE
FTVAPPQ_USER_ID

FTVAPPQ_SURROGATE_ID
FTVAPPQ_DATA_ORIGIN
FTVAPPQ_VERSION
FTVAPPQ_VPDI_CODE


# Resources

https://learncodeshare.net/2015/06/02/basic-crud-operations-using-cx_oracle/
https://learncodeshare.net/2015/06/26/insert-crud-using-cx_oracle/

https://cx-oracle.readthedocs.io/en/latest/user_guide/sql_execution.html

https://www.foxinfotech.in/2018/09/how-to-call-oracle-function-in-python.html

https://www.oracletutorial.com/oracle-basics/oracle-delete/

https://www.oracletutorial.com/python-oracle/updating-data/

https://www.programcreek.com/python/example/55416/cx_Oracle.connect